# Fipkart-products Recommendation Systems
## Content-Based Recommender Systems

In [1]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

products = pd.read_csv('data/flipkart_com-ecommerce_sample.csv')[:100]
products = products[['uniq_id', 'description', 'retail_price']].dropna()


target = products.iloc[:,0].values

count_vector = CountVectorizer()
count_array = count_vector.fit_transform(products['description'])

tfidf        = TfidfTransformer()
tfidf_vector = tfidf.fit_transform(count_array)
tfitf_array  = tfidf_vector.toarray()

new_feature = np.concatenate([tfitf_array, np.reshape(products['retail_price'], [-1, 1])], axis=1)

/Users/tuck/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [2]:
np.reshape(products['retail_price'], [-1, 1])

/Users/tuck/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


array([[   999.],
       [ 32157.],
       [   999.],
       [   699.],
       [   220.],
       [   430.],
       [  1199.],
       [ 32157.],
       [   699.],
       [  1199.],
       [  1724.],
       [  2299.],
       [   999.],
       [   699.],
       [   999.],
       [ 32157.],
       [   899.],
       [  2499.],
       [ 32157.],
       [   110.],
       [  2400.],
       [  1500.],
       [   190.],
       [  1299.],
       [  3199.],
       [   499.],
       [   699.],
       [  2199.],
       [   799.],
       [   999.],
       [   499.],
       [   499.],
       [  1000.],
       [  1200.],
       [   350.],
       [   899.],
       [   999.],
       [   899.],
       [  1999.],
       [  1499.],
       [  6999.],
       [  2299.],
       [   999.],
       [   899.],
       [  2099.],
       [  6500.],
       [  9990.],
       [  1399.],
       [   899.],
       [  1299.],
       [  8999.],
       [   899.],
       [ 10000.],
       [   899.],
       [   899.],
       [  

In [3]:
new_feature

array([[     0.,      0.,      0., ...,      0.,      0.,    999.],
       [     0.,      0.,      0., ...,      0.,      0.,  32157.],
       [     0.,      0.,      0., ...,      0.,      0.,    999.],
       ..., 
       [     0.,      0.,      0., ...,      0.,      0.,   3999.],
       [     0.,      0.,      0., ...,      0.,      0.,   1799.],
       [     0.,      0.,      0., ...,      0.,      0.,   1415.]])

In [20]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
clf.fit(new_feature, target)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False)

## Predict with keyword + price

In [21]:
keyword = 'Women\'s Cycling'
price = 1000
count_array = count_vector.transform([keyword])
p_tfidf_vector = tfidf.transform(count_array)
p_tfitf_array  = p_tfidf_vector.toarray()
p_new_feature = np.concatenate([p_tfitf_array, np.reshape([price], [-1, 1])], axis=1)

In [34]:
probs = clf.predict_proba(p_new_feature)
best_n = np.argsort(probs, axis=1)[:1, -3:]

array([35, 96, 53])

In [35]:
targets = []


In [36]:
products[products['uniq_id'] == targets]

ValueError: Arrays were different lengths: 97 vs 0